# Filtrado de tablas para obtener información de actividades solo de la asignatura IP


El objetivo de este script es el de filtrar las tablas que recogen información de las tareas entregadas para obtener solo aquellas relativas  al curso de ip, y así evitar tener que hacer un join para concatenar usuario con la tarea y otro para unir estas tuplas con solo las tareas pertenecientes a un curso. 

A la hora de concatenar, solo nos quedaremos con los atributos de utilidad de cada tabla.

## Configuración 

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import os

# Configuración
curso_ip = 8683
ruta_origen = "/home/carlos/Documentos/TFG/spark-workspace/data/raw"
ruta_destino = "/home/carlos/Documentos/TFG/spark-workspace/data/intermediate/concatenados"
os.makedirs(ruta_destino, exist_ok=True)

# Crear sesión Spark
spark = SparkSession.builder \
    .appName("Filtrado datos curso IP") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "8") \
    .getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/12 20:11:26 WARN Utils: Your hostname, carlos-Modern-15-A11SB, resolves to a loopback address: 127.0.1.1; using 172.17.0.1 instead (on interface docker0)
25/06/12 20:11:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/12 20:11:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/06/12 20:11:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/06/12 20:11:27 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


## Filtrado de assignments y concatenación de entregas y notas

Tras realizar esta operación, nos quedaremos con una única tabla  `assign_submission_grade_cmi` que va a tener información de  todas las tareas entregadas por los estudiantes de ip, junto con la calificación que obtuvieron en ellas.

 Es clave comprender que, dado que haremos un `inner join` entre las tareas de la asignatura, con las entregas asociadas a ellas, solo se guardarán como resultado de la unión aquellas tareas que tienen asociadas al menos una entrega, que son las que tienen validez para las métricas que se pretenden calcular. 

In [ ]:
# Leer datos de asignaciones y envíos
from math import trunc


assign = spark.read.parquet(f"{ruta_origen}/assign_cmi.parquet")
submissions = spark.read.parquet(f"{ruta_origen}/assign_submission_cmi.parquet")


# Filtrar tabla para quedarnos solo con las tareas de ip , y quedarnos solo con los campos necesarios
assign_ip = assign.filter(col("course") == curso_ip).select(
    "id", "duedate", "allowsubmissionsfromdate", "name"
)
# Concatenamos tareas con sus entregas de tal modo que no se obtendrán las tareas que no tienen asociada ninguna entrega.
submissions_ip = (
    submissions.join(assign_ip, submissions.assignment == assign_ip.id, "inner")
    .withColumnRenamed("timemodified", "timesubmitted")
    .drop("id")
)



number_of_submissions = submissions_ip.count()

print(f"Número de envíos de tareas del curso {curso_ip}: {number_of_submissions}")


# Concatenar submissions con grades, para tener información junta de entregas y notas.
grades = spark.read.parquet(f"{ruta_origen}/assign_grades_cmi.parquet")
grades_filtered = grades.select("userid", "assignment", "grade")

assign_submission_grade_cmi = submissions_ip.join(
    grades_filtered, on=["userid", "assignment"], how="left"
).drop("status")

#Comprobamos el número de tuplas tras la unión, para verificar si aumenta el número de tuplas,
#lo cual podría significar que puede haber asociadas varias notas para una misma entrega.
number_of_assign_submission_grade = assign_submission_grade_cmi.count()
print(f"Número de envíos de tareas con notas del curso {curso_ip}: {number_of_assign_submission_grade}\n")

print("Comprobamos que no exista más de una entrega por usuario para una misma tarea.")

num_subByUser = (
    assign_submission_grade_cmi
    .groupBy("userid", "assignment")
    .count()
    .filter(col("count") > 1)
)

num_subByUser_count = num_subByUser.count()
print(f"Número de usuarios con más de una entrega por tarea: {num_subByUser_count} \n")
print("Nombre de los usuarios con más de una entrega por tarea, junto a la tarea en la que tienen más de una entrega y el número de entregas")
num_subByUser.show(num_subByUser_count, truncate=False)

print("Primera cinco filas del dataframe generado:")
assign_submission_grade_cmi.show(5, truncate=False)

assign_submission_grade_cmi.printSchema()

# # Escribir el dataframe a parquet 
# assign_submission_grade_cmi.write.mode("overwrite").parquet(f"{ruta_destino}/assign_submission_grade_cmi.parquet")
# print("assign_submission_grade_cmi.parquet creado :)")



Número de envíos de tareas del curso 8683: 1309
Número de envíos de tareas con notas del curso 8683: 1309

Comprobamos que no exista más de una entrega por usuario para una misma tarea.
Número de usuarios con más de una entrega por tarea: 4 

Nombre de los usuarios con más de una entrega por tarea, junto a la tarea en la que tienen más de una entrega y el número de entregas
+----------------------------------------------------------------+----------+-----+
|userid                                                          |assignment|count|
+----------------------------------------------------------------+----------+-----+
|5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9|108682    |29   |
|5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9|108640    |62   |
|5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9|109226    |61   |
|5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9|113721    |58   |
+----------------------------------

25/06/12 20:11:45 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers


Como podemos apreciar, el número de tuplas que tiene el dataframe producto de la unión de tareas con entregas es igual al valor devuelto al calcularlo para el dataframe resultante de la unión del anterior con las notas por cada usuario y tarea. Por lo tanto, se ha verificado con éxito que no existe más de una nota publicada para cada entrega de una tarea.

Por otra parte, se ha podido apreciar que, mientras que en la mayroría de casos solo hay una sola entrega asociada a un usuario, hay cuatro usuarios que presentan muchísimas más entregas, algo que resulta muy extraño.

## Exploración de los datos 


A continuación, se van a explorar  los datos obtenidos en la anterior celda. En primer lugar, las aulas pueden tener muchas actividades publicadas, pero que permanecen durante el curso ocultas para los alumnos dado que son de utilidad al profesorado pero no se pretende que las realicen los alumnos. Por lo tanto, para nuestras métricas solo se deberán considerar aquellas que hayan sido realmente publicadas durante el curso.

Para ello, analizaremos , del total de tareas que están publicadas en el aula, cuales de ellas tienen registrada al menos una entrega, dado que esto significará que estuvo visible.

Al  calcularse la unión de las entregas con las actividades  para dar lugar al dataframe `submissions_ip` se hizo un `inner join`, aquellas tareas que no tuvieran asociadas ninguna entrega no cumplieron la condición de join y por lo tanto no fueron devueltas en la operación. 

Por este motivo, para comprobar qué tareas realmente tuvieron alguna entrega nos bastará con contar el número de tuplas de ese dataframe, quedandonos solo con los valores distintos del identificador de tarea, para evitar contar el número de entregas de cada tarea.

In [3]:

from pyspark.sql.functions import from_unixtime, date_format

# Número total de tareas del curso
num_total_tareas_ip = assign_ip.select("id").distinct().count()

# Número de tareas con al menos una entrega asociada
num_tareas_con_entregas_ip = assign_submission_grade_cmi.select("assignment").distinct().count() 

print(f"Número total de tareas: {num_total_tareas_ip}")
print(f"Número de tareas con al menos una entrega: {num_tareas_con_entregas_ip} \n")

print("===========================================\n")

print("Identificador, nombre y fecha de entrega de todas las tareas")

assign_ip_f = assign_ip.withColumn(
      "duedate_formatted", date_format(from_unixtime(col("duedate")), "yy/MM/dd")
)
assign_ip_f.select("id", "name", "duedate_formatted").orderBy("name").show(100, truncate=False)

print("\n")
print("Identificador, nombre y fecha de entrega de las tareas con al menos una entrega")

assign_submission_grade_cmi_F = assign_submission_grade_cmi.withColumn(
      "duedate_formatted", date_format(from_unixtime(col("duedate")), "yy/MM/dd")
)
assign_submission_grade_cmi_F.select(
      "assignment", "name", "duedate_formatted"
).dropDuplicates(["assignment"]).orderBy("name").show(100, truncate=False)
    

Número total de tareas: 34
Número de tareas con al menos una entrega: 14 


Identificador, nombre y fecha de entrega de todas las tareas
+------+--------------------------------------------------------------------------------------------------------------------------+-----------------+
|id    |name                                                                                                                      |duedate_formatted|
+------+--------------------------------------------------------------------------------------------------------------------------+-----------------+
|7592  |Actividad 00. Actualización del perfil en el campus virtual                                                               |23/09/25         |
|96123 |Calificación y comentarios de la actividad 02                                                                             |70/01/01         |
|107688|Entrega actividad 07                                                                                     

Tras ver detenidamente la salida, podemos apreciar algo muy extraño.

La gran mayoría de las tareas del aula, no tienen asociada ninguna entrega en el curso que se está considerando. Se tiene que estudiar a qué se puede debrer esto, dado que resulta muy extraño que solo se tenga información de entregas en la primera tarea del curso, y de las entregas del proyecto de la asignatura, pero no de ninguna de las que se producen en los meses de octubre y noviembre. Además, se puede observar que hay registradas entregas en tareas que en principio, finalizaban en años anteriores.

Por ejemplo, fijemonos en todas las entregas que hizo un estudiante  aleatorio a lo largo de un curso.


In [4]:
as_sub_grad_submitFormatted = assign_submission_grade_cmi_F.withColumn(
    "timesubmitted_formatted", date_format(from_unixtime(col("timesubmitted")), "dd/MM/yy")
)
(as_sub_grad_submitFormatted.filter(col("userid") == "df7ce50a753bbb53b8e1d9eaf6a3432fa82b4e47e002142d2cf924451f3b39b1" )
      .select("userid", "assignment","name", "grade", "timesubmitted_formatted", "duedate_formatted")
      .orderBy("assignment", "timesubmitted_formatted")
      .show(100, truncate=False)
 )

+----------------------------------------------------------------+----------+-----------------------------------------------------------------------+--------+-----------------------+-----------------+
|userid                                                          |assignment|name                                                                   |grade   |timesubmitted_formatted|duedate_formatted|
+----------------------------------------------------------------+----------+-----------------------------------------------------------------------+--------+-----------------------+-----------------+
|df7ce50a753bbb53b8e1d9eaf6a3432fa82b4e47e002142d2cf924451f3b39b1|7592      |Actividad 00. Actualización del perfil en el campus virtual            |2.00000 |12/09/23               |23/09/25         |
|df7ce50a753bbb53b8e1d9eaf6a3432fa82b4e47e002142d2cf924451f3b39b1|107688    |Entrega actividad 07                                                   |-1.00000|19/11/23               |23/11/20      

 ## Unión con los usuarios con permiso de alumno y matriculados en la asignatura

A coninuación, vamos a verificar si hay cambios en los datos cuando hacemos un join con la tabla en la que se guarda el id de los alumnos que estaban inscritos al curso el año pasado. Quizás, de este modo se eliminan los usuarios que tenian múltiples entregas para una sola tarea. Si bien la lógica nos dice, que al haber filtrado los datos de las tareas para quedarnos solo con los de la asignatura ip, y que por lo tanto los datos recogidos de las entregas deberían coincidir con los alumnos de la asignatura, así nos aseguramos que no se cualen datos sucios.

In [6]:
alumnos_ip = spark.read.parquet(f"{ruta_origen}/alumnos_ip_cmi.parquet")

print("\n")

# Unir los datos de alumnos con las entregas y notas
as_sub_grad_alumnnos = assign_submission_grade_cmi.join(alumnos_ip, on="userid", how="inner")
print (f"Número de tuplas de la tabla tras unir con los alumnos matriculados: {as_sub_grad_alumnnos.count()}\n")

print("Comprobamos si existe ahora el usuario que tenía multiples entregas sobre varias tareas")

as_sub_grad_alumnnos.filter(col("userid") == "5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9").select("userid").distinct().show() 

print("Ahora, podemos comprobar cuantos usuarios han dejado de ser considerados tras unir con los alumnos matriculados, es decir, cuantos usuarios de los que había presente en la primera tabla realmente \n eran alumnos de la asignatura ese curso\n")
num_users_1 = assign_submission_grade_cmi.select("userid").distinct().count()
num_users_2 = as_sub_grad_alumnnos.select("userid").distinct().count()
print(f"Número de usuarios presentes en la primera tabla sin filtrar por alumnos matriculados : {num_users_1}\n")
print(f"Número de usuarios presentes en la segunda tabla filtrada por alumnos matriculados : {num_users_2}\n")




Número de tuplas de la tabla tras unir con los alumnos matriculados: 1071

Comprobamos si existe ahora el usuario que tenía multiples entregas sobre varias tareas
+------+
|userid|
+------+
+------+

Ahora, podemos comprobar cuantos usuarios han dejado de ser considerados tras unir con los alumnos matriculados, es decir, cuantos usuarios de los que había presente en la primera tabla realmente 
 eran alumnos de la asignatura ese curso

Número de usuarios presentes en la primera tabla sin filtrar por alumnos matriculados : 209

Número de usuarios presentes en la segunda tabla filtrada por alumnos matriculados : 201



En primer lugar, podemos apreciar que, tras realizar el join han desaparecido varias tuplas, dado que en la tabla inicialmente había 1309 en un inicio, y ahora han desaparecido varias, dado que tras el join hay 1071. En concreto, han desaparecido 8 usuarios de la tabla, y dado que uno de ellos hemos podido comprobar que era el usuario que tenía multiples entregas sobre varias tareas, tiene sentido que haya bajado  el número de tuplas de la tabla tras desaparecer estos usuarios.

Además, podemos observar que, ha desaparecido el usuario que tenía multiples entregas en varias de las tareas al unir solo con los alumnos matrículados, quizás por que haya desaparecido al filtrar por alumnos, por que se guardase también algun detalle relativo a las interacciones de los profesores con las entregas.

Finalmente, podemos comprobar si el número de tareas que tienen entregas permanece constante tras esta unión, o si ha cambiado y ha desaparecido alguna. 


In [7]:
print(f"Número de tareas con al menos una entrega previamente a la unión con los alumnos matriculados: {num_tareas_con_entregas_ip}")
print(f"Número de tareas con al menos una entrega tras la unión con los alumnos matriculados: {as_sub_grad_alumnnos.select('assignment').distinct().count()}\n")
print("===========================================\n")
print("Identificador, nombre y fecha de entrega de las tareas con al menos una entrega antes de unir con los alumnos matriculados")
assign_submission_grade_cmi_F.select(
      "assignment", "name", "duedate_formatted"
).dropDuplicates(["assignment"]).orderBy("name").show(100, truncate=False)
print("\n")
print("Identificador, nombre y fecha de entrega de las tareas con al menos una entrega tras unir con los alumnos matriculados")
as_sub_grad_alumnnos_F = as_sub_grad_alumnnos.withColumn(
      "duedate_formatted", date_format(from_unixtime(col("duedate")), "dd/MM/yy")
)
as_sub_grad_alumnnos_F.select(
      "assignment", "name", "duedate_formatted"
).dropDuplicates(["assignment"]).orderBy("name").show(100, truncate=False)
print("\n")




Número de tareas con al menos una entrega previamente a la unión con los alumnos matriculados: 14
Número de tareas con al menos una entrega tras la unión con los alumnos matriculados: 10


Identificador, nombre y fecha de entrega de las tareas con al menos una entrega antes de unir con los alumnos matriculados
+----------+-----------------------------------------------------------------------+-----------------+
|assignment|name                                                                   |duedate_formatted|
+----------+-----------------------------------------------------------------------+-----------------+
|7592      |Actividad 00. Actualización del perfil en el campus virtual            |23/09/25         |
|96123     |Calificación y comentarios de la actividad 02                          |70/01/01         |
|107688    |Entrega actividad 07                                                   |23/11/20         |
|108682    |Entrega de la actividad 08 - Proyecto - Parejas - NO VALE 

Finalmente, se puede ver cómo, han desaparecido del dataframe tras la unión con los alumnos aquellas tareas que eran de otros años pero qeu seguramente fueran entregadas en convocatorias extraordinarias por estudiantes de años anteriores, dado que tras filtrar por solo aquellos que estaban matriculados en el aula virtual ese año, solo nos queda información de tareas cuya fecha de entrega coincide con los meses en los que se imparte el curso.

Sin embargo, cabe destacar que sigue sin haber rastro de ninguna tarea realizada ni entregada en octubre, y casi tampoco en noviembre, los cuales en principio son meses muy activos de tareas en ese curso.

## Verificación de tareas en octubre


En último lugar, vamos a asegurarnos de que no haya tareas realmente en el aula en el mes de octubre, ni ninguna entrega en esa fecha

In [11]:
from pyspark.sql.functions import to_date
from pyspark.sql.functions import to_timestamp


# Convertir la columna 'duedate_formatted' a tipo fecha
assign_ip_f = assign_ip_f.withColumn(
    "duedate_date", to_date(col("duedate_formatted"), "yy/MM/dd")
)

print("Vamos a explorar cuantas tareas de ip tienen fijada la fecha de entrega en algún momento de octubre\n")
filtered_assignments = assign_ip_f.filter(
    (col("duedate_date") > "2023-10-01") & (col("duedate_date") < "2023-10-31")
)

filtered_assignments.select("id", "name", "duedate_date").show(truncate=False)

print(
    "Ahora vamos a comprobar si existen entregas que hayan sido entregadas durante el mes de octubre de 2023\n"
)


# Convertir la columna 'timesubmitted' a tipo timestamp
sub_ip_date = submissions_ip.withColumn(
    "timesubmitted_date", to_timestamp(from_unixtime(col("timesubmitted")), "yyyy-MM-dd HH:mm:ss")
)

# Filtrar las entregas cuya fecha sea posterior a '01/11/23' y anterior a '30/11/23'
filtered_submissions = sub_ip_date.filter(
    (col("timesubmitted_date") > "2023-10-01 00:00:00") &
    (col("timesubmitted_date") < "2023-10-31 23:59:59")
)

filtered_submissions.select("userid", "assignment", "timesubmitted_date", "name").drop_duplicates(["assignment"]).show(100, truncate=False)

Vamos a explorar cuantas tareas de ip tienen fijada la fecha de entrega en algún momento de octubre

+---+----+------------+
|id |name|duedate_date|
+---+----+------------+
+---+----+------------+

Ahora vamos a comprobar si existen entregas que hayan sido entregadas durante el mes de octubre de 2023



+----------------------------------------------------------------+----------+-------------------+-----------------------------------------------------------+
|userid                                                          |assignment|timesubmitted_date |name                                                       |
+----------------------------------------------------------------+----------+-------------------+-----------------------------------------------------------+
|091af124e119a447c7f6594fb2f7c4fbb678f669966db01e3f62c26eedb220af|7592      |2023-10-01 19:46:08|Actividad 00. Actualización del perfil en el campus virtual|
+----------------------------------------------------------------+----------+-------------------+-----------------------------------------------------------+



Se puede apreciar que solo hay entregas de las tareas `7592` en octubre, siendo esta la tarea solo la actividad 0.

# Escritura


In [14]:
as_sub_grad_alumnnos.printSchema()

as_sub_grad_alumnnos.orderBy("userid").show(200, truncate=False)

as_sub_grad_alumnnos.groupBy("userid", "assignment").count().orderBy("userid").show(200, truncate=False)

as_sub_grad_alumnnos.write.mode("overwrite").parquet(f"{ruta_destino}/assign_sub_grad_al_ip.parquet")

root
 |-- userid: string (nullable = true)
 |-- assignment: long (nullable = true)
 |-- timesubmitted: long (nullable = true)
 |-- duedate: long (nullable = true)
 |-- allowsubmissionsfromdate: long (nullable = true)
 |-- name: string (nullable = true)
 |-- grade: string (nullable = true)

+----------------------------------------------------------------+----------+-------------+----------+------------------------+-----------------------------------------------------------------------+--------+
|userid                                                          |assignment|timesubmitted|duedate   |allowsubmissionsfromdate|name                                                                   |grade   |
+----------------------------------------------------------------+----------+-------------+----------+------------------------+-----------------------------------------------------------------------+--------+
|006b0e7bd07cec05e0952cb61c30893f6d30d7962f9efc99d0f041f6fadcc320|7592      |16971